### Очистка и подготовка данных

Данные не всегда представляют собой идеально организованный набор наблюдений без каких-либо пропусков или аномалий Обычно данные содержат в себе кучу мусора, который необходимо почистить, да и вообще сами данные порой лучше воспринимать критически, для того чтобы затем привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.



![](https://sun9-34.userapi.com/c205324/v205324823/352a5/O_5h-Lk70I4.jpg)

### 2 главные проблемы при подготовке данных: пропущенные значения и неправильная типизация


In [1]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/alnagaev/landing/master/train.csv')

In [2]:
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [3]:
df.info()
#показывает в том числе сколько пропущенных значений

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Очистка данных порой предполагает избавление от неиспользуемых сущностей/столбцов

In [4]:
# Удаляем столбцы, в которых нет ценной для нас информации
df.drop(['PassengerId', 'Name', 'Ticket'], axis = 1, inplace = True)

In [5]:
# Округляем стоимость билета до двух знаков после запятой
df['Fare'] = round(df['Fare'], 2)

In [6]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.25,NaN,S
1,1,1,female,38.0,1,0,71.28,C85,C
2,1,3,female,26.0,0,0,7.92,NaN,S
3,1,1,female,35.0,1,0,53.10,C123,S
4,0,3,male,35.0,0,0,8.05,NaN,S


 Посчитаем пропущенные значения

In [7]:
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

Nan = not a number 

### Пропущенные значения можно удалить, но скорее так делать не следует

In [8]:
df = df.dropna()

#можно настраивать и изменять способ удаления данных, например с помощью параметра thresh=2, который оставит строки с боллее чем 2 непустыми значениями

### Пропущенные значения можно заменять на что-то другое

In [9]:
#Способ 1: Константа 

df['Age'].fillna(0)

1      38.0
3      35.0
6      54.0
10      4.0
11     58.0
       ... 
871    47.0
872    33.0
879    56.0
887    19.0
889    26.0
Name: Age, Length: 183, dtype: float64

In [10]:
# Способ 2: Средние арифметические
df['Age'].fillna(df['Age'].median())

1      38.0
3      35.0
6      54.0
10      4.0
11     58.0
       ... 
871    47.0
872    33.0
879    56.0
887    19.0
889    26.0
Name: Age, Length: 183, dtype: float64

In [11]:
# Способ 2: Cредние арифметические в зависимости от класса каюты
age_1 = df[df['Pclass'] == 1]['Age'].mean()
age_2 = df[df['Pclass'] == 2]['Age'].mean()
age_3 = df[df['Pclass'] == 3]['Age'].mean()

In [12]:
def fill_age(row):
    if pd.isnull(row['Age']):
        if row['Pclass'] == 1:
            return age_1
        if row['Pclass'] == 2:
            return age_2
        return age_3
    return row['Age']

In [13]:
df.apply(fill_age, axis = 1)

1      38.0
3      35.0
6      54.0
10      4.0
11     58.0
       ... 
871    47.0
872    33.0
879    56.0
887    19.0
889    26.0
Length: 183, dtype: float64

https://pandas.pydata.org/pandas-docs/version/0.16.2/generated/pandas.DataFrame.interpolate.html

https://stackoverflow.com/questions/19966018/pandas-filling-missing-values-by-mean-in-each-group

In [14]:
df.groupby('Pclass')['Age'].apply(lambda x:x.fillna(x.mean()))

1      38.0
3      35.0
6      54.0
10      4.0
11     58.0
       ... 
871    47.0
872    33.0
879    56.0
887    19.0
889    26.0
Name: Age, Length: 183, dtype: float64

На всякий случай проверяем равны ли полученные объекты

In [15]:
(df.apply(fill_age, axis = 1)).equals(df.groupby('Pclass')['Age'].apply(lambda x:x.fillna(x.mean())))

True

In [16]:
df.groupby('Pclass').agg('mean')

,Survived,Age,SibSp,Parch,Fare
Pclass,,,,,
1,0.670886,37.591266,0.487342,0.468354,88.683354
2,0.800000,25.266667,0.400000,0.400000,18.444667
3,0.500000,21.000000,0.200000,0.700000,11.028000


Принцип действия **groupby**
![](https://i.stack.imgur.com/sgCn1.jpg)

- Разделение данных по группам по определенному критерию

- Применение к каждой группе определенной функции

- Сложение результатов в единую структуру

### Создаем новый столбец с информацией о том, был ли пассажир на борту один или с родственниками
Столбец должен содержать значение "alone", если он был на борту один (без супруга/супруги, братьев, сестер, детей и родителей) и значение "not alone", если пассажир путешествовал с кем-то из родственников

In [17]:
# Способ 1: с помощью именной функции и apply
def alone_check(row):
    if row['SibSp'] or row['Parch'] > 0:
        return 'not_alone'
    return 'alone'
    
df['Alone'] = df.apply(alone_check, axis=1)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Alone
1,1,1,female,38.0,1,0,71.28,C85,C,not_alone
3,1,1,female,35.0,1,0,53.10,C123,S,not_alone
6,0,1,male,54.0,0,0,51.86,E46,S,alone
10,1,3,female,4.0,1,1,16.70,G6,S,not_alone
11,1,1,female,58.0,0,0,26.55,C103,S,alone
...,...,...,...,...,...,...,...,...,...,...
871,1,1,female,47.0,1,1,52.55,D35,S,not_alone
872,0,1,male,33.0,0,0,5.00,B51 B53 B55,S,alone
879,1,1,female,56.0,0,1,83.16,C50,C,not_alone
887,1,1,female,19.0,0,0,30.00,B42,S,alone


In [18]:
# Способ 2: с помощью lambda-функции
df.drop('Alone', axis=1, inplace = True)

df['Alone'] = df.apply(lambda x: 'not_alone' if x['SibSp'] or x['Parch'] > 0 else 'alone', axis = 1)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Alone
1,1,1,female,38.0,1,0,71.28,C85,C,not_alone
3,1,1,female,35.0,1,0,53.10,C123,S,not_alone
6,0,1,male,54.0,0,0,51.86,E46,S,alone
10,1,3,female,4.0,1,1,16.70,G6,S,not_alone
11,1,1,female,58.0,0,0,26.55,C103,S,alone
...,...,...,...,...,...,...,...,...,...,...
871,1,1,female,47.0,1,1,52.55,D35,S,not_alone
872,0,1,male,33.0,0,0,5.00,B51 B53 B55,S,alone
879,1,1,female,56.0,0,1,83.16,C50,C,not_alone
887,1,1,female,19.0,0,0,30.00,B42,S,alone


### Строим сводную таблицу

In [19]:
df.pivot_table(values = 'Pclass', index = 'Survived', columns = 'Alone', aggfunc = 'count')

Alone,alone,not_alone
Survived,,
0,31,29
1,48,75


### Дополнительные приемы работы с числами

In [20]:
new_df = pd.DataFrame({'col1': [1,2,3,4,5], 'col2': '2,5 4,5 3.3 1,5 2,9'.split()})

new_df

,col1,col2
0,1,"2,5"
1,2,"4,5"
2,3,3.3
3,4,"1,5"
4,5,"2,9"


In [21]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
col1    5 non-null int64
col2    5 non-null object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


In [22]:
new_df['col2'] = new_df['col2'].apply(lambda x: x.replace(',', '.'))


In [23]:
new_df['col2'] = pd.to_numeric(new_df['col2'])

In [24]:
new_df['col2'] = new_df['col2'].astype('float64', inplace = True)

In [25]:
new_df['col2'] = new_df['col2'].apply(lambda x: float(x))

#### Если нужно заменить сразу много столбцов

In [26]:
strNum = pd.DataFrame({'col1': '56,5 2,5 3.3 1,5 2,9'.split(), 'col2': '2,5 4,5 3.3 1,5 2,9'.split(), 
                       'col3': '7,6 4,5 3.3 7,5 1,9'.split()})
strNum

,col1,col2,col3
0,"56,5","2,5","7,6"
1,"2,5","4,5","4,5"
2,3.3,3.3,3.3
3,"1,5","1,5","7,5"
4,"2,9","2,9","1,9"


In [27]:
for i in list(strNum):
    strNum[i] = strNum[i].apply(lambda x: x.replace(',', '.')).astype('float64')



### Работа с датами

In [28]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2016-weather-data-seattle.csv')

df.tail(10)

,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC
24371,12/22/2015,8,6.0,3.0
24372,12/23/2015,5,4.0,3.0
24373,12/24/2015,6,4.0,2.0
24374,12/25/2015,5,4.0,2.0
24375,12/26/2015,4,2.0,0.0
24376,12/27/2015,4,3.0,2.0
24377,12/28/2015,5,3.0,2.0
24378,12/29/2015,7,4.0,1.0
24379,12/30/2015,6,2.0,-1.0
24380,12/31/2015,6,2.0,-2.0


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24381 entries, 0 to 24380
Data columns (total 4 columns):
Date                 24381 non-null object
Max_TemperatureC     24381 non-null int64
Mean_TemperatureC    24376 non-null float64
Min_TemperatureC     24380 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 762.0+ KB


In [30]:
df.isna().sum()

Date                 0
Max_TemperatureC     0
Mean_TemperatureC    5
Min_TemperatureC     1
dtype: int64

In [31]:
df['Date'] = pd.to_datetime(df['Date'])
df['Date']

0       1948-01-01
1       1948-01-02
2       1948-01-03
3       1948-01-04
4       1948-01-05
           ...    
24376   2015-12-27
24377   2015-12-28
24378   2015-12-29
24379   2015-12-30
24380   2015-12-31
Name: Date, Length: 24381, dtype: datetime64[ns]

In [32]:
df['Date'].dt.year

0        1948
1        1948
2        1948
3        1948
4        1948
         ... 
24376    2015
24377    2015
24378    2015
24379    2015
24380    2015
Name: Date, Length: 24381, dtype: int64

### Создание новых временнЫх признаков

In [33]:
df['current_time'] = pd.to_datetime('today').strftime("%m/%d/%Y")



In [34]:
df['current_time'] = pd.to_datetime(df['current_time'])

In [35]:
df['timedelta'] = (df['current_time']-df['Date']).astype('timedelta64[Y]')
df

,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,current_time,timedelta
0,1948-01-01,10,8.0,7.0,2020-01-27,72.0
1,1948-01-02,6,4.0,3.0,2020-01-27,72.0
2,1948-01-03,7,4.0,2.0,2020-01-27,72.0
3,1948-01-04,7,4.0,2.0,2020-01-27,72.0
4,1948-01-05,7,3.0,0.0,2020-01-27,72.0
...,...,...,...,...,...,...
24376,2015-12-27,4,3.0,2.0,2020-01-27,4.0
24377,2015-12-28,5,3.0,2.0,2020-01-27,4.0
24378,2015-12-29,7,4.0,1.0,2020-01-27,4.0
24379,2015-12-30,6,2.0,-1.0,2020-01-27,4.0


### Удаление дупликатов

In [36]:
data = {"Name": ["James", "Alice", "Phil", "James"],
"Age": [24, 28, 40, 24],
"Sex": ["Male", "Female", "Male", "Male"]}

df = pd.DataFrame(data)
df


,Name,Age,Sex
0,James,24,Male
1,Alice,28,Female
2,Phil,40,Male
3,James,24,Male


In [37]:
df[:4]

,Name,Age,Sex
0,James,24,Male
1,Alice,28,Female
2,Phil,40,Male
3,James,24,Male


In [38]:
df.drop_duplicates()

,Name,Age,Sex
0,James,24,Male
1,Alice,28,Female
2,Phil,40,Male


### Строковые методы очистки данных

In [39]:
df = pd.read_csv('https://raw.githubusercontent.com/alnagaev/reni_land/master/test.csv')
df.head(10)

,id,Дата рождения,Вакансия,Технология,Локация,З/п в валюте найма
0,1,18.04.1994,Инженер-программист 2 категории,JS,Санкт-Петербург,"140 000,00 ₽"
1,2,27.03.1993,Разработчик информационных систем,.NET,Москва,"130 000,00 ₽"
2,3,24.11.1983,Java Developer,Java,Санкт-Петербург,"200 000,00 ₽"
3,4,09.06.1989,Архитектор ПО,Java,Москва,"250 000,00 ₽"
4,5,03.10.1978,QA Automation Engineer,QA,Санкт-Петербург,"172 413,00 ₽"
5,6,21.08.1973,Big Data Developer,Big Data,Санкт-Петербург,"280 000,00 ₽"
6,7,10.10.1991,JS Developer,JS,Москва,"129 000,00 ₽"
7,8,16.08.1982,Разработчик мобильных приложений,Android,Москва,"185 000,00 ₽"
8,9,16.05.1988,Инженер-программист 2 категории,.NET,Санкт-Петербург,"160 000,00 ₽"
9,10,01.12.1990,Java Engineer,Java,Краков; Польша,"9 500,00 zł"


Создание новых признаков

In [40]:
df['salary'] = df['З/п в валюте найма'].apply(lambda x: int(''.join([s for s in list(x) if s.isdigit()]))/100)
df['currency'] = df['З/п в валюте найма'].apply(lambda x: ''.join([s for s in list(x) if not s.isdigit()]).replace(',', ''))
df.head(10)

,id,Дата рождения,Вакансия,Технология,Локация,З/п в валюте найма,salary,currency
0,1,18.04.1994,Инженер-программист 2 категории,JS,Санкт-Петербург,"140 000,00 ₽",140000.0,₽
1,2,27.03.1993,Разработчик информационных систем,.NET,Москва,"130 000,00 ₽",130000.0,₽
2,3,24.11.1983,Java Developer,Java,Санкт-Петербург,"200 000,00 ₽",200000.0,₽
3,4,09.06.1989,Архитектор ПО,Java,Москва,"250 000,00 ₽",250000.0,₽
4,5,03.10.1978,QA Automation Engineer,QA,Санкт-Петербург,"172 413,00 ₽",172413.0,₽
5,6,21.08.1973,Big Data Developer,Big Data,Санкт-Петербург,"280 000,00 ₽",280000.0,₽
6,7,10.10.1991,JS Developer,JS,Москва,"129 000,00 ₽",129000.0,₽
7,8,16.08.1982,Разработчик мобильных приложений,Android,Москва,"185 000,00 ₽",185000.0,₽
8,9,16.05.1988,Инженер-программист 2 категории,.NET,Санкт-Петербург,"160 000,00 ₽",160000.0,₽
9,10,01.12.1990,Java Engineer,Java,Краков; Польша,"9 500,00 zł",9500.0,zł


**Приведем список технологий к единообразию на примере JS**

In [41]:
df['Технология'] = df['Технология'].replace('JavaScript', 'JS')

In [42]:
df[df["Технология"].isin(["JS", 'Java'])]

,id,Дата рождения,Вакансия,Технология,Локация,З/п в валюте найма,salary,currency
0,1,18.04.1994,Инженер-программист 2 категории,JS,Санкт-Петербург,"140 000,00 ₽",140000.00,₽
2,3,24.11.1983,Java Developer,Java,Санкт-Петербург,"200 000,00 ₽",200000.00,₽
3,4,09.06.1989,Архитектор ПО,Java,Москва,"250 000,00 ₽",250000.00,₽
6,7,10.10.1991,JS Developer,JS,Москва,"129 000,00 ₽",129000.00,₽
9,10,01.12.1990,Java Engineer,Java,Краков; Польша,"9 500,00 zł",9500.00,zł
10,11,04.02.1991,Инженер-программист,JS,Москва,"170 000,00 ₽",170000.00,₽
12,13,04.11.1988,JavaScript Engineer,JS,Краков; Польша,"10 500,00 zł",10500.00,zł
13,14,09.04.1990,Software Engineer,JS,Вроцлав; Польша,"9 500,00 zł",9500.00,zł
15,16,30.08.1982,Java Developer,Java,Москва,"180 000,00 ₽",180000.00,₽
17,18,28.08.1995,JS Developer,JS,Москва,"160 000,00 ₽",160000.00,₽


In [43]:
df.query('Технология == "JS" | Технология == "Java"')

,id,Дата рождения,Вакансия,Технология,Локация,З/п в валюте найма,salary,currency
0,1,18.04.1994,Инженер-программист 2 категории,JS,Санкт-Петербург,"140 000,00 ₽",140000.00,₽
2,3,24.11.1983,Java Developer,Java,Санкт-Петербург,"200 000,00 ₽",200000.00,₽
3,4,09.06.1989,Архитектор ПО,Java,Москва,"250 000,00 ₽",250000.00,₽
6,7,10.10.1991,JS Developer,JS,Москва,"129 000,00 ₽",129000.00,₽
9,10,01.12.1990,Java Engineer,Java,Краков; Польша,"9 500,00 zł",9500.00,zł
10,11,04.02.1991,Инженер-программист,JS,Москва,"170 000,00 ₽",170000.00,₽
12,13,04.11.1988,JavaScript Engineer,JS,Краков; Польша,"10 500,00 zł",10500.00,zł
13,14,09.04.1990,Software Engineer,JS,Вроцлав; Польша,"9 500,00 zł",9500.00,zł
15,16,30.08.1982,Java Developer,Java,Москва,"180 000,00 ₽",180000.00,₽
17,18,28.08.1995,JS Developer,JS,Москва,"160 000,00 ₽",160000.00,₽


Метод **str.contains**

In [44]:
df[df['Вакансия'].str.contains("Engineer", na=False)]

,id,Дата рождения,Вакансия,Технология,Локация,З/п в валюте найма,salary,currency
4,5,03.10.1978,QA Automation Engineer,QA,Санкт-Петербург,"172 413,00 ₽",172413.0,₽
9,10,01.12.1990,Java Engineer,Java,Краков; Польша,"9 500,00 zł",9500.0,zł
11,12,11.02.1982,.NET Engineer,.NET,Гданьск; Польша,"12 000,00 zł",12000.0,zł
12,13,04.11.1988,JavaScript Engineer,JS,Краков; Польша,"10 500,00 zł",10500.0,zł
13,14,09.04.1990,Software Engineer,JS,Вроцлав; Польша,"9 500,00 zł",9500.0,zł
18,19,02.02.1972,Senior Software Engineer,BI,Санкт-Петербург,"200 000,00 ₽",200000.0,₽
19,20,21.09.1985,QA Engineer,QA,Санкт-Петербург,"135 000,00 ₽",135000.0,₽
37,38,03.04.1990,JavaScript Engineer,JS,Краков; Польша,"13 000,00 zł",13000.0,zł
38,39,10.06.1988,Senior Software Engineer,Java,София; Болгария,"lev6 446,00",6446.0,lev
39,40,17.02.1992,Test Automation Engineer,QA,Будапешт; Венгрия,"Ft650 000,00",650000.0,Ft
